# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

* Multi agent system are great but they really shine when you get the ability for these agents to talk to each other

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

### Multi-agent collaboration techniques
In CrewAI, there are several multi-agent collaboration techniques, each designed to facilitate different kinds of interactions and workflows among agents. Here’s an overview of the main techniques:

#### Sequential Process:

- Description: In this process, tasks are executed one after another in a defined sequence. Each agent completes its assigned task before the next agent begins.

- Use Case: This technique is ideal for scenarios where the output of one task is necessary for the next, ensuring a step-by-step progression.

#### Parallel Process:

- Description: Here, multiple agents work on their tasks simultaneously. This process is used when tasks are independent of each other and can be executed in parallel without dependencies.

- Use Case: Suitable for increasing efficiency by leveraging multiple agents at the same time.

#### Hierarchical Process: 

- Description: In a hierarchical process, agents are organized in a hierarchy where higher-level agents(Manager) can delegate tasks to lower-level agents. This allows for a top-down management structure within the crew.

- Use Case: Best used in complex projects where tasks need to be broken down into sub-tasks and managed by different levels of agents.

#### Collaboration with Memory:

- Description: This technique allows agents to share a collective memory, enabling them to access and build on each other's work. Memory can be used to store intermediate results, decisions, and other relevant information.

- Use Case: Ideal for ongoing projects where context and past decisions need to be considered in future tasks.

#### Kickoff Async:

- Description: This method allows tasks to be kicked off asynchronously, meaning agents can start working on their tasks at different times, not necessarily in a strict sequence.

- Use Case: Useful in dynamic environments where tasks may start at different times based on the availability of data or resources.

#### Kickoff for Each:

- Description: This is used to kickoff tasks for each agent in a crew, ensuring that all agents start their tasks concurrently.

- Use Case: Suitable for scenarios where all agents need to work simultaneously on different aspects of a project.


* Sequential
* Hierarchical(You can create Agent Manger Yourself) 
* Asynchronous

![alt text](image.png)

![alt text](<Screenshot (245).png>)

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [3]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-4o-mini", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [2024-09-02 10:51:27][DEBUG]: == Working Agent: Crew Manager
 [2024-09-02 10:51:27][INFO]: == Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst. I will provide thorough context regarding the task requirements, including the need for statistical modeling and machine learning techniques to identify trends and predict market movements. 

Action: Delegate work to coworker  
Action Input: {"task":"Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.","context":"The task involves gathering real-time market data for AAPL, which includes price changes, trading volume, and news sentiment. You 

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(result.raw)

### Risks Associated with Proposed Trading Strategies for AAPL

#### 1. Trading Two-Day Highs/Lows

**Risk Exposure:**
- **Market Volatility:** AAPL's current beta of 1.24 indicates moderate volatility. If the broader market experiences significant movements, AAPL could be affected, leading to potential losses beyond the stop-loss levels.
- **Price Breakout Failure:** If AAPL fails to break above the two-day high of $230.40 or drops below the two-day low of $227.48, traders could face losses if the price does not reverse in a favorable direction.

**Mitigation Strategies:**
- **Adjust Stop-Loss Levels:** Consider tightening stop-loss levels to minimize potential losses, particularly in volatile conditions.
- **Monitor Market Trends:** Keep an eye on overall market trends and events that may influence AAPL's stock price, such as earnings reports or macroeconomic indicators.

#### 2. Trading Ranges During Low Volatility

**Risk Exposure:**
- **False Breakouts:** In a low volatility environment, prices may hover near support or resistance levels without making a decisive move, leading to potential losses if trades are initiated on false signals.
- **Liquidity Issues:** Low volatility can often correlate with lower trading volume, which may exacerbate price swings and lead to slippage when executing trades.

**Mitigation Strategies:**
- **Set Wider Stop-Loss Levels:** Given the risk of false breakouts, consider using wider stop-loss levels to accommodate potential price fluctuations while still maintaining acceptable risk levels.
- **Use Limit Orders:** Implement limit orders to ensure better control over entry and exit prices, especially during low volatility periods.

#### 3. GAP-FILL Strategy

**Risk Exposure:**
- **Earnings Report Impact:** With an upcoming earnings report, AAPL’s stock may experience significant price gaps. If the report is unfavorable, it could lead to sharp declines, triggering stop-loss orders and exacerbating losses.
- **Market Sentiment Changes:** External factors such as economic data releases or competitor performance can shift market sentiment quickly, impacting AAPL's stock price unpredictably.

**Mitigation Strategies:**
- **Pre-Earnings Caution:** Reduce position sizes leading up to the earnings report to limit exposure to adverse price movements.
- **Utilize Options:** Consider using options strategies (e.g., puts for protection) to hedge against potential downside risks associated with earnings volatility.

### Conclusion
Overall, trading AAPL carries inherent risks, especially in light of current market conditions and recent performance trends. It is crucial to remain vigilant, adjust strategies as necessary, and maintain a disciplined approach to risk management. Continuous monitoring of market indicators and AAPL-specific news will be essential in adapting trading strategies to mitigate risks effectively.